# 🤖 Nano-Analyst: Fine-Tuning Pipeline

**Portfolio Project: Private On-Device SQL Agent**

This notebook fine-tunes Llama-3-8B on the Spider dataset for Text-to-SQL generation using QLoRA and Unsloth.

---

## ⚙️ Setup Checklist

Before running:
1. ✅ **GPU Enabled**: Runtime → Change runtime type → T4 GPU
2. ✅ **High RAM**: Runtime → Change runtime type → High-RAM
3. ✅ **Data Ready**: Upload `train.json` and `validation.json` from Step 1

---

**Estimated Time:**
- Setup: 5 minutes
- Training: 2-3 hours (T4 GPU)
- Total: ~3 hours

## 📋 Step 1: Verify GPU

In [ ]:
# Check GPU availability
!nvidia-smi

import torch
print(f"\n{'='*70}")
print("GPU VERIFICATION")
print(f"{'='*70}")
print(f"✓ PyTorch version: {torch.__version__}")
print(f"✓ CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"✓ GPU: {torch.cuda.get_device_name(0)}")
    print(f"✓ VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("❌ NO GPU DETECTED!")
    print("   Go to: Runtime → Change runtime type → Hardware accelerator → GPU")
print(f"{'='*70}")

## 📦 Step 2: Install Dependencies

**Critical:** Install Unsloth for 2x faster training and 50% less memory usage.

In [ ]:
%%capture
# Install Unsloth (optimized for Colab)
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

# Install training dependencies
!pip install --no-deps trl peft accelerate bitsandbytes

In [ ]:
# Verify installation
import unsloth
print(f"✓ Unsloth version: {unsloth.__version__}")
print("✓ All dependencies installed successfully!")

## 💾 Step 3: Mount Google Drive & Upload Data

**Important:** This saves your model to Google Drive so it persists after the session ends.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Create project directory in Drive
import os
project_dir = '/content/drive/MyDrive/nano-analyst'
os.makedirs(f"{project_dir}/data/processed", exist_ok=True)
os.makedirs(f"{project_dir}/models", exist_ok=True)

print(f"✓ Project directory created: {project_dir}")
print("\n📂 Next: Upload your data files to:")
print(f"   {project_dir}/data/processed/")
print("\n   Required files:")
print("   - train.json")
print("   - validation.json")

### 📤 Upload Data Files

**Option A:** Use Colab file browser (left sidebar) to upload to `/content/drive/MyDrive/nano-analyst/data/processed/`

**Option B:** Run data preparation here:

In [ ]:
# Option B: Prepare data directly in Colab
!pip install datasets

# Download and prepare Spider dataset
import json
from datasets import load_dataset
from pathlib import Path

print("Downloading Spider dataset from HuggingFace...")
spider = load_dataset("spider")

def format_to_alpaca(example):
    """Format Spider example to Alpaca instruction format."""
    # Build schema string
    schema_parts = []
    if 'db_table_names' in example and 'db_column_names' in example:
        from collections import defaultdict
        table_cols = defaultdict(list)
        
        for col_data in example['db_column_names']:
            table_idx = col_data['table_id']
            col_name = col_data['column_name']
            if table_idx >= 0 and table_idx < len(example['db_table_names']):
                table_cols[table_idx].append(col_name)
        
        for table_idx, table_name in enumerate(example['db_table_names']):
            cols = table_cols.get(table_idx, [])
            if cols:
                cols_str = ",\n  ".join([f"{c} TEXT" for c in cols])
                schema_parts.append(f"CREATE TABLE {table_name} (\n  {cols_str}\n);")
    
    schema = "\n\n".join(schema_parts) if schema_parts else "-- Schema not available --"
    
    return {
        "instruction": "You are an expert SQL generator. Given a database schema and a natural language question, generate the correct SQL query to answer the question. Output only the SQL query without any explanations.",
        "input": f"Database Schema:\n{schema}\n\nQuestion: {example['question']}",
        "output": example['query']
    }

# Process train split
print("\nFormatting training data...")
train_formatted = [format_to_alpaca(ex) for ex in spider['train']]

# Split into train/val
import random
random.seed(42)
random.shuffle(train_formatted)
split_idx = int(len(train_formatted) * 0.9)
train_subset = train_formatted[:split_idx]
val_subset = train_formatted[split_idx:]

# Save
data_dir = Path(project_dir) / "data" / "processed"
with open(data_dir / "train.json", 'w') as f:
    json.dump(train_subset, f, indent=2)
with open(data_dir / "validation.json", 'w') as f:
    json.dump(val_subset, f, indent=2)

print(f"\n✓ Saved {len(train_subset)} training examples")
print(f"✓ Saved {len(val_subset)} validation examples")
print(f"✓ Data ready at: {data_dir}")

## 🧠 Step 4: Load Model & Add LoRA Adapters

In [ ]:
from unsloth import FastLanguageModel
import torch

# Configuration
max_seq_length = 1536  # Schema + Question + SQL
dtype = None  # Auto-detect
load_in_4bit = True  # Critical for T4 GPU

# Load model (this downloads ~5GB on first run)
print("Loading Llama-3-8B-Instruct (4-bit quantized)...")
print("This may take 2-3 minutes...\n")

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/llama-3-8b-Instruct-bnb-4bit",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

print("✓ Model loaded!\n")

# Add LoRA adapters
print("Adding LoRA adapters...")
model = FastLanguageModel.get_peft_model(
    model,
    r=32,  # LoRA rank (higher = more capacity)
    lora_alpha=64,  # Scaling factor
    lora_dropout=0.05,
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj"
    ],
    bias="none",
    use_gradient_checkpointing="unsloth",  # 2x memory savings
    random_state=42,
)

# Print parameter counts
trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
total = sum(p.numel() for p in model.parameters())
print(f"\n✓ LoRA adapters configured")
print(f"  Trainable params: {trainable:,} ({100*trainable/total:.2f}%)")
print(f"  Total params: {total:,}")

## 📊 Step 5: Prepare Training Data

In [ ]:
import json
from datasets import Dataset
from pathlib import Path

# Load data
data_dir = Path(project_dir) / "data" / "processed"

with open(data_dir / "train.json", 'r') as f:
    train_data = json.load(f)

with open(data_dir / "validation.json", 'r') as f:
    val_data = json.load(f)

print(f"✓ Loaded {len(train_data)} training examples")
print(f"✓ Loaded {len(val_data)} validation examples")

# Format into Llama-3 chat template
def format_prompt(example):
    """Format example using Llama-3-Instruct chat template."""
    return f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>

{example['instruction']}<|eot_id|><|start_header_id|>user<|end_header_id|>

{example['input']}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

{example['output']}<|eot_id|>"""

print("\nFormatting prompts...")
train_prompts = [format_prompt(ex) for ex in train_data]
val_prompts = [format_prompt(ex) for ex in val_data]

# Create datasets
train_dataset = Dataset.from_dict({"text": train_prompts})
val_dataset = Dataset.from_dict({"text": val_prompts})

print("✓ Datasets formatted and ready!")

# Show sample
print("\n" + "="*70)
print("SAMPLE TRAINING EXAMPLE")
print("="*70)
print(train_prompts[0][:500] + "...")
print("="*70)

## 🚀 Step 6: Configure Training & Start Fine-Tuning

**This is the main training step - it will take 2-3 hours on T4 GPU.**

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

# Output directory (in Google Drive for persistence)
output_dir = f"{project_dir}/models/nano-analyst-v1"

# Training arguments (optimized for T4)
training_args = TrainingArguments(
    output_dir=output_dir,
    
    # Batch size (T4 limit)
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=8,  # Effective batch = 16
    
    # Training schedule
    num_train_epochs=3,
    learning_rate=2e-4,
    warmup_steps=100,
    
    # Logging & evaluation
    logging_steps=10,
    eval_steps=250,
    save_steps=500,
    evaluation_strategy="steps",
    save_strategy="steps",
    
    # Optimization
    optim="adamw_8bit",  # Memory-efficient optimizer
    weight_decay=0.01,
    max_grad_norm=1.0,
    lr_scheduler_type="cosine",
    
    # Mixed precision
    fp16=True,  # T4 doesn't support bf16
    
    # Checkpointing
    save_total_limit=3,  # Keep only 3 best checkpoints
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    
    # Reporting
    logging_dir=f"{output_dir}/logs",
    report_to="none",  # Disable WandB for simplicity
)

# Create trainer
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    args=training_args,
    packing=False,  # Don't pack sequences (important for SQL)
)

# Disable cache during training
model.config.use_cache = False

print("="*70)
print("TRAINING CONFIGURATION")
print("="*70)
print(f"Total epochs: 3")
print(f"Batch size: 2")
print(f"Gradient accumulation: 8")
print(f"Effective batch size: 16")
print(f"Learning rate: 2e-4")
print(f"Max sequence length: 1536")
print(f"\nEstimated time: 2-3 hours on T4")
print("="*70)

# Start training
print("\n🚀 Starting training...\n")
trainer.train()

## 💾 Step 7: Save Fine-Tuned Model

In [ ]:
# Save LoRA adapters (lightweight - only ~200MB)
lora_dir = f"{output_dir}/lora_adapters"

model.save_pretrained(lora_dir)
tokenizer.save_pretrained(lora_dir)

print(f"✓ LoRA adapters saved to: {lora_dir}")
print(f"\nFile size: ~200MB (much smaller than full 16GB model!)")
print(f"\nYour model is saved to Google Drive and will persist after this session ends.")

## 🧪 Step 8: Test the Model (Quick Inference Check)

In [ ]:
# Enable inference mode
FastLanguageModel.for_inference(model)

# Test query
test_schema = """CREATE TABLE singers (
  singer_id INTEGER,
  name TEXT,
  age INTEGER,
  country TEXT
);"""

test_question = "How many singers are from France?"

prompt = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are an expert SQL generator. Given a database schema and a natural language question, generate the correct SQL query to answer the question. Output only the SQL query without any explanations.<|eot_id|><|start_header_id|>user<|end_header_id|>

Database Schema:
{test_schema}

Question: {test_question}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

"""

inputs = tokenizer([prompt], return_tensors="pt").to("cuda")

print("Testing fine-tuned model...\n")
print(f"Question: {test_question}")
print(f"\nExpected SQL: SELECT COUNT(*) FROM singers WHERE country = 'France'\n")

outputs = model.generate(
    **inputs,
    max_new_tokens=128,
    temperature=0.1,
    top_p=0.9,
    do_sample=True
)

generated = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
sql = generated.split("assistant")[-1].strip()

print(f"Generated SQL: {sql}")
print("\n✓ Model is working! Ready for Step 3 (SQL Agent with RAG)")

## 🎉 Training Complete!

### What You've Accomplished:

✅ Fine-tuned Llama-3-8B on 6,000+ Text-to-SQL examples

✅ Used QLoRA for efficient 4-bit training

✅ Leveraged Unsloth for 2x faster training

✅ Saved model to Google Drive (persists after session)

### Next Steps:

1. **Step 3**: Build the SQLAgent class with RAG retrieval
2. **Step 4**: Run evaluation benchmarks (Execution Accuracy)
3. **Portfolio**: Deploy as a Streamlit app or API

### Model Location:

Your fine-tuned model is saved in:
```
/content/drive/MyDrive/nano-analyst/models/nano-analyst-v1/lora_adapters/
```

**File size:** ~200MB (LoRA adapters only)

---

**Questions?** Check the troubleshooting guide in `STEP2_TRAINING_GUIDE.md`